In [ ]:
# Import necessary libraries
import boto3
import pandas as pd
import io
import json
from datetime import datetime

# Define the SageMaker endpoint name
endpoint_name = 'simple-linear-regression-endpoint'  # Replace with your actual SageMaker endpoint name
region_name = 'us-east-1'  # Specify your AWS region

# Function to invoke the SageMaker endpoint
def invoke_endpoint(payload):
    client = boto3.client('sagemaker-runtime', region_name=region_name)
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(payload),
        ContentType='application/json'
    )
    result = json.loads(response['Body'].read().decode())
    return result

# Test the function with a sample payload
payload = {"data": [[0.00632, 18.00, 2.31, 0.0, 0.538, 6.575, 65.2, 4.0900, 1, 296.0, 15.3, 396.90, 4.98]]}  # Example input
result = invoke_endpoint(payload)
print("Inference result:", result)

# Convert the result to a pandas DataFrame
inference_data = {
    "input": [payload["data"]],
    "prediction": [result]
}
df = pd.DataFrame(inference_data)

# Save the DataFrame as a CSV file
csv_buffer = io.StringIO()
df.to_csv(csv_buffer, index=False)

# Upload the CSV file to S3
s3_client = boto3.client('s3', region_name=region_name)
bucket_name = 'justin-automation-output'
csv_file_name = f'inferences_{datetime.now().strftime("%Y%m%d%H%M%S")}.csv'

s3_client.put_object(
    Bucket=bucket_name,
    Key=csv_file_name,
    Body=csv_buffer.getvalue()
)

print(f"Inference results saved to S3 bucket '{bucket_name}' with file name '{csv_file_name}'.")
